<a href="https://colab.research.google.com/github/alvinicdf-sketch/2026/blob/main/%E9%A7%90%E5%A4%96%E4%BA%BA%E5%93%A1%E5%90%8D%E5%86%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#若遇到❌ Ngrok 啟動失敗:解決方法
#1. 方案一: 在colabd工作列打開執行階段>重新啟動工作階段，再run一次code，重複個兩三次就會好，我也不知道為啥。(已經把初始化時間拉到40秒，應該會穩一點)
#2. 或是ERROR結果是跳Alreay online, 會有一個連結，點那個連結就好
#3. 方案二: 登入ngrok網站，到左欄位traffic inspector, Agents把正在啟用的線路刪除


In [ ]:
!pip install uv -qq
# A 區塊: 使用 uv 安裝 Streamlit、Ngrok 與 Excel 處理套件
!uv pip install --system streamlit Pillow pyngrok openpyxl pandas matplotlib -qq

print("所有必要的 Python 套件已透過 uv 安裝完成。")

所有必要的 Python 套件已透過 uv 安裝完成。


In [ ]:
# @title
# B 區塊: 掛載 Drive、建立 Main.py 檔案並啟動 Streamlit 服務
from google.colab import drive
import os
import subprocess
import time
from pyngrok import ngrok, conf
import sys
import threading
import re
import pandas as pd
import matplotlib.font_manager as fm
import glob
import signal
import urllib.request

# --- 檔案路徑設定 ---
DRIVE_ROOT = '/content/drive/My Drive/駐外人員名冊colab'
EXCEL_FILE = os.path.join(DRIVE_ROOT, '全部的駐外人員統整表(滾動式更新-閱覽用).xlsx')
SCORES_FILE = os.path.join(DRIVE_ROOT, '駐外歷年考核成績(滾動式更新) 未包含新員工.xlsx')
LOGO_FILE = os.path.join(DRIVE_ROOT, 'ICDF LOGO.png')

# --- Matplotlib 中文字型設定 (在 Colab 環境中執行) ---
print("\n開始設定 Matplotlib 中文字型...")
try:
    # 1. 安裝中文支援字型 Noto Sans CJK
    subprocess.run(["apt-get", "install", "-y", "fonts-noto-cjk"],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

    # 2. 清理 Matplotlib 字型快取
    cache_dir = os.path.join(os.path.expanduser('~'), '.cache', 'matplotlib')
    cache_files = glob.glob(os.path.join(cache_dir, 'fontlist-v*.json'))

    deleted_count = 0
    for f in cache_files:
        try:
            os.remove(f)
            deleted_count += 1
        except OSError:
            pass

    if deleted_count > 0:
        print(f"✅ Matplotlib 快取檔案已清除 ({deleted_count} 個)。")
    else:
        print("✅ 中文字型 Noto Sans CJK 已安裝。")
except Exception as e:
    print(f"❌ 中文字型設定失敗: {e}")

# --- 下載並設定 Ngrok ---
print("\n正在下載並設定 Ngrok...")
NGROK_PATH = "/content/ngrok"

try:
    # 下載 Ngrok (Linux AMD64版本)
    if not os.path.exists(NGROK_PATH):
        print("下載 Ngrok...")
        ngrok_url = "https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz"
        urllib.request.urlretrieve(ngrok_url, "/content/ngrok.tgz")

        # 解壓縮
        subprocess.run(["tar", "-xzf", "/content/ngrok.tgz", "-C", "/content/"], check=True)

        # 設定執行權限
        os.chmod(NGROK_PATH, 0o755)
        print("✅ Ngrok 下載並設定完成。")
    else:
        print("✅ Ngrok 已存在。")
        # 確保有執行權限
        os.chmod(NGROK_PATH, 0o755)

except Exception as e:
    print(f"❌ Ngrok 設定失敗: {e}")
    sys.exit(1)

# --- Ngrok 認證金鑰 ---
AUTH_TOKEN = "38EF6xMMHRMz4uvdJgzbz8XUghk_55QYr39r9SSVMqMsBbxSc"
try:
    # 設定 Ngrok 路徑
    conf.get_default().ngrok_path = NGROK_PATH
    ngrok.set_auth_token(AUTH_TOKEN)
    print("✅ Ngrok 認證金鑰已設定。")
except Exception as e:
    print(f"❌ 設定 Ngrok 認證金鑰時發生錯誤: {e}")

# 清除舊的 Ngrok 連線
try:
    ngrok.kill()
    print("✅ 已清除所有舊的 Ngrok 服務。")
except Exception:
    pass

# --- 掛載 Google Drive ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("✅ Drive 已掛載。")

os.makedirs(DRIVE_ROOT, exist_ok=True)
os.chdir(DRIVE_ROOT)

print(f"目前工作目錄已切換到: {os.getcwd()}")

# --- 檔案存在性檢查 ---
print("\n--- 檔案存在性檢查 ---")
FILES_TO_CHECK = {
    '統整表': EXCEL_FILE,
    '考核表': SCORES_FILE,
    'LOGO圖': LOGO_FILE
}

all_files_exist = True
for name, path in FILES_TO_CHECK.items():
    if os.path.exists(path):
        print(f"✅ {name} 檔案已找到: {path}")
    else:
        print(f"❌ 警告：找不到 {name} 檔案！請確認此檔案在您的 Google Drive 中: {path}")
        all_files_exist = False

if not all_files_exist:
    print("\n🚨 由於缺少關鍵檔案，Streamlit 應用程式可能無法正常顯示數據。請先將所有檔案放置到 Drive 中的指定路徑。")

# --- 1. 寫入 main_app.py 檔案 ---
print("\n開始寫入 main_app.py...")

main_app_template = """
import streamlit as st
from PIL import Image
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from matplotlib import rcParams
import matplotlib.font_manager as fm

# 檔案路徑
LOGO_PATH = __LOGO_FILE_PATH__
EXCEL_FILE = __EXCEL_FILE_PATH__
SCORES_FILE = __SCORES_FILE_PATH__
DRIVE_ROOT = __DRIVE_ROOT_PATH__

# --- Matplotlib 中文設定 (使用 Noto Sans CJK TC) ---
def setup_matplotlib_font():
    # 尋找 Noto Sans CJK TC 字型的路徑
    font_path_tc = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'

    if os.path.exists(font_path_tc):
        fm.fontManager.addfont(font_path_tc)
        try:
            prop = fm.FontProperties(fname=font_path_tc)
            font_name = prop.get_name()
            plt.rcParams['font.sans-serif'] = [font_name, 'Microsoft JhengHei', 'Arial Unicode MS']
        except Exception:
            plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'Microsoft JhengHei', 'Arial Unicode MS']
    else:
        plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'Microsoft JhengHei', 'Arial Unicode MS']

    plt.rcParams['axes.unicode_minus'] = False

setup_matplotlib_font()

# --- 主題感知顏色設定函數 ---
def get_theme_colors():
    base = st.get_option("theme.base")
    if base == 'dark':
        text_color = 'white'
        pie_label_color_inside = '#F0F0F0'
        pie_legend_facecolor = '#333333'
        chart_line_color = 'white'
    else:
        text_color = 'black'
        pie_label_color_inside = '#1A1A1A'
        pie_legend_facecolor = '#e0e0e0'
        chart_line_color = 'black'

    return text_color, pie_label_color_inside, pie_legend_facecolor, chart_line_color

# 設定寬屏
st.set_page_config(layout="wide", initial_sidebar_state="collapsed")

st.markdown('''
<style>
    .fixed-button-container {
        position: fixed;
        top: 10px;
        left: 20px;
        z-index: 1000;
        width: auto;
    }
    .fixed-button-container button {
        height: 38px;
        padding: 0px 16px;
        font-size: 14px;
    }
    input[type=number]::-webkit-inner-spin-button,
    input[type=number]::-webkit-outer-spin-button {
      -webkit-appearance: none;
      margin: 0;
    }
    input[type=number] {
      -moz-appearance: textfield;
    }
</style>
''', unsafe_allow_html=True)

def switch_page(target_page):
    st.session_state.page = target_page

def toggle_filter(state):
    st.session_state.show_filter = state

def display_title(title_text):
    st.markdown(f"<h1 style='text-align: center;'>{title_text}</h1>", unsafe_allow_html=True)

@st.cache_data
def load_data():
    df_staff = pd.DataFrame()
    df_scores = pd.DataFrame()

    if not os.path.exists(EXCEL_FILE):
        st.error(f"❌ 嚴重錯誤：找不到統整表檔案！請確認檔案在路徑中：{EXCEL_FILE}")
        return df_staff, df_scores

    if not os.path.exists(SCORES_FILE):
        st.warning(f"⚠️ 警告：找不到考核表檔案！人員資料頁面的考核分數圖表將無法顯示：{SCORES_FILE}")

    try:
        df_staff = pd.read_excel(EXCEL_FILE)
        if os.path.exists(SCORES_FILE):
            df_scores = pd.read_excel(SCORES_FILE, sheet_name='工作表2')

        df_staff.columns = df_staff.columns.str.strip()
        df_scores.columns = df_scores.columns.str.strip()

        if '員工姓名' in df_staff.columns:
            df_staff['員工姓名'] = df_staff['員工姓名'].astype(str).str.strip()
        if '姓名' in df_scores.columns:
            df_scores['姓名'] = df_scores['姓名'].astype(str).str.strip()

        return df_staff, df_scores
    except Exception as e:
        st.error(f"讀取 Excel 檔案時發生錯誤，請檢查檔案格式是否正確: {e}")
        return pd.DataFrame(), pd.DataFrame()


def apply_filters(df, filters):
    df_filtered = df.copy()

    for key, values in filters.items():
        if not values or (isinstance(values, list) and len(values) == 0):
            continue

        if key == '年齡':
            try:
                min_age = float('-inf')
                max_age = float('inf')

                if values['below']:
                    max_age = min(max_age, values['below'])
                if values['between_start'] and values['between_end']:
                    min_age = max(min_age, values['between_start'])
                    max_age = min(max_age, values['between_end'])
                if values['above']:
                    min_age = max(min_age, values['above'])

                if min_age != float('-inf') or max_age != float('inf'):
                    df_filtered = df_filtered[
                        (df_filtered['年齡'] > min_age) &
                        (df_filtered['年齡'] < max_age)
                    ]
            except KeyError:
                pass
            except Exception as e:
                st.warning(f"年齡篩選條件設定錯誤：{e}")
        else:
            try:
                df_filtered = df_filtered[df_filtered[key].isin(values)]
            except KeyError:
                pass

    return df_filtered

def draw_score_chart(df_scores, staff_name):
    text_color, _, _, chart_line_color = get_theme_colors()

    score_cols_pattern = re.compile(r'^(1\\d{2}|200)年\\s*核定分數$')
    score_cols = [col for col in df_scores.columns if score_cols_pattern.match(col.strip())]

    try:
        score_cols.sort(key=lambda x: int(re.search(r'(\\d+)年', x).group(1)))
    except AttributeError:
        pass

    staff_name_clean = staff_name.strip()
    try:
        staff_data = df_scores[df_scores['姓名'] == staff_name_clean]
    except KeyError:
        return

    if staff_data.empty:
        st.info(f"找不到 {staff_name_clean} 的考核分數資料。")
        return

    scores = staff_data[score_cols].iloc[0].apply(pd.to_numeric, errors='coerce')
    scores = scores.dropna()

    if scores.empty:
        st.info(f"{staff_name_clean} 沒有可用的考核分數資料。")
        return

    years_full = [col.split('年')[0] + '年' for col in scores.index]
    years_only_number = [col.split('年')[0] for col in scores.index]

    fig, ax = plt.subplots(figsize=(3, 1.5))
    ax.plot(years_full, scores.values, marker='o', linestyle='-', color=chart_line_color, markersize=3)

    ax.set_title("Annual Performance Score (年度考核分數)", color=text_color, fontsize=7)
    ax.set_xlabel("年度", color=text_color, fontsize=6)
    ax.set_ylabel("分數", color=text_color, fontsize=6)
    ax.grid(True, linestyle='--', alpha=0.6, color='gray')
    ax.set_xticks(years_full)
    ax.set_xticklabels(years_only_number)

    Y_OFFSET = 1.5
    for i, score_value in enumerate(scores.values):
        ax.text(x=years_full[i], y=score_value + Y_OFFSET, s=f'{int(score_value)}',
                ha='center', va='center', fontsize=6, color=text_color)

    if not scores.empty:
        y_max = scores.max() + Y_OFFSET + 2
        ax.set_ylim(scores.min() - 5, y_max)

    ax.tick_params(axis='x', colors=text_color, labelsize=6)
    ax.tick_params(axis='y', colors=text_color, labelsize=6)
    ax.spines['bottom'].set_color(text_color)
    ax.spines['left'].set_color(text_color)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    fig.patch.set_alpha(0.0)
    ax.set_facecolor('none')

    st.pyplot(fig, use_container_width=False)
    plt.close(fig)

def get_multiselect_keys():
    return ['性別', '語系', '駐在國', '種類', '職稱', '職等', '學歷']

def reset_filters():
    for key in get_multiselect_keys():
        st.session_state.filters[key] = []
        filter_key_str = f'filter_{key}'
        if filter_key_str in st.session_state:
            st.session_state[filter_key_str] = []

    st.session_state.age_below = None
    st.session_state.age_between_start = None
    st.session_state.age_between_end = None
    st.session_state.age_above = None

    for age_key in ['age_b_input', 'age_bs_input', 'age_be_input', 'age_a_input']:
        if age_key in st.session_state:
            del st.session_state[age_key]

    st.session_state.run_filter = False
    st.success("篩選條件已重製，請重新設定。")
    st.rerun()

def show_filter_sidebar():
    st.sidebar.markdown("<h2>人員篩選器</h2>", unsafe_allow_html=True)

    if st.sidebar.button("❌ 關閉篩選器", key="btn_close_filter", use_container_width=True):
        toggle_filter(False)
        st.rerun()
        return

    options = {
        '性別': ['男', '女'],
        '語系': ['英語', '西文'],
        '駐在國': ['烏克蘭', '瓜地馬拉', '薩爾瓦多', '貝里斯', '海地', '聖文森', '聖克里斯多福及尼維斯', '聖露西亞', '巴布亞紐幾內亞', '泰國', '印尼', '吐瓦魯', '帛琉', '馬紹爾', '斐濟', '菲律賓', '土耳其', '巴林', '沙烏地阿拉伯', '約旦', '立陶宛', '史瓦帝尼', '索馬利蘭', '坦尚尼亞', '巴拉圭'],
        '種類': ['華語教學', '漁業', '農業', '公衛醫療', '畜牧獸醫', '資訊', '其他專家', '經貿投資', '技職', '農業資訊'],
        '職稱': ['華語教師', '技師', '技術員', '計畫經理', '專家', '團長'],
        '職等': ['技術員', '技師', '專家', '資深專家'],
        '學歷': ['大學', '碩士']
    }

    if 'filters' not in st.session_state:
        st.session_state.filters = {}

    st.sidebar.markdown("---")

    for key, opts in options.items():
        with st.sidebar.expander(f"選擇 {key}", expanded=False):
            st.session_state.filters[key] = st.multiselect(
                f"選擇 {key}", opts,
                default=st.session_state.filters.get(key, []),
                key=f'filter_{key}'
            )

    with st.sidebar.expander("選擇 年齡", expanded=False):
        if 'age_below' not in st.session_state: st.session_state.age_below = None
        if 'age_between_start' not in st.session_state: st.session_state.age_between_start = None
        if 'age_between_end' not in st.session_state: st.session_state.age_between_end = None
        if 'age_above' not in st.session_state: st.session_state.age_above = None

        st.markdown("##### 低於 (Below)")
        st.session_state.age_below = st.number_input(
            "年齡低於", min_value=18, max_value=100, step=None, key='age_b_input',
            value=st.session_state.age_below, format="%i"
        )

        st.markdown("##### 區間 (Between)")
        col_start, col_end = st.columns(2)
        with col_start:
            st.session_state.age_between_start = st.number_input(
                "年齡介於 (起)", min_value=18, max_value=100, step=None, key='age_bs_input',
                value=st.session_state.age_between_start, format="%i"
            )
        with col_end:
            st.session_state.age_between_end = st.number_input(
                "年齡介於 (終)", min_value=18, max_value=100, step=None, key='age_be_input',
                value=st.session_state.age_between_end, format="%i"
            )

        st.markdown("##### 高於 (Above)")
        st.session_state.age_above = st.number_input(
            "年齡高於", min_value=18, max_value=100, step=None, key='age_a_input',
            value=st.session_state.age_above, format="%i"
        )

    st.session_state.filters['年齡'] = {
        'below': st.session_state.age_below,
        'between_start': st.session_state.age_between_start,
        'between_end': st.session_state.age_between_end,
        'above': st.session_state.age_above
    }

    st.sidebar.markdown("---")
    col_ok, col_reset = st.sidebar.columns(2)

    with col_ok:
        if st.button("✔ 確定", key="btn_filter_ok", use_container_width=True, type="primary"):
            st.session_state.page = 'search'
            st.session_state.show_filter = False
            st.session_state.run_filter = True
            st.rerun()

    with col_reset:
        st.button("條件重製", key="btn_filter_reset", use_container_width=True, on_click=reset_filters)

def home_page():
    text_color, pie_label_color_inside, pie_legend_facecolor, _ = get_theme_colors()

    display_title("國合會駐外人員名冊")

    try:
        if os.path.exists(LOGO_PATH.replace("'", "")):
            logo = Image.open(LOGO_PATH.replace("'", ""))
            col1, col2, col3 = st.columns([1.5, 1, 1.5])
            with col2:
                st.image(logo, width=350)
        else:
            st.warning("⚠️ 找不到 ICDF LOGO.png，請確認檔案位置。")
    except Exception as e:
        st.error(f"載入 LOGO 失敗: {e}")

    df_staff, _ = load_data()

    if not df_staff.empty:
        st.markdown("<div style='margin-top: 40px;'></div>", unsafe_allow_html=True)

        chart_col1, chart_col2 = st.columns(2)

        with chart_col1:
            st.markdown("#### 👥 性別分佈 (Gender Distribution)")
            gender_counts = df_staff['性別'].value_counts()
            total = len(df_staff)

            fig_gender, ax_gender = plt.subplots(figsize=(1.8, 1.8))
            colors = ['#4A90E2', '#E24A8D']

            labels_inside = [
                f"{gender_counts.get('男', 0)} ({gender_counts.get('男', 0)/total*100:.1f}%)",
                f"{gender_counts.get('女', 0)} ({gender_counts.get('女', 0)/total*100:.1f}%)"
            ]

            gender_data = [gender_counts.get('男', 0), gender_counts.get('女', 0)]

            wedges, texts = ax_gender.pie(
                gender_data,
                labels=labels_inside,
                colors=colors,
                startangle=90,
                textprops={'color': pie_label_color_inside, 'fontsize': 9, 'weight': 'bold'}
            )

            ax_gender.legend(['男', '女'], loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                          fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

            ax_gender.set_facecolor('none')
            fig_gender.patch.set_alpha(0.0)

            st.pyplot(fig_gender, use_container_width=False)
            plt.close(fig_gender)

        with chart_col2:
            st.markdown("#### 📊 種類分佈 (Type Distribution)")
            type_counts = df_staff['種類'].value_counts()

            fig_type, ax_type = plt.subplots(figsize=(1.8, 1.8))
            colors_type = plt.cm.Set3(range(len(type_counts)))

            percentages = [(count/total*100) for count in type_counts.values]

            wedges, texts = ax_type.pie(
                type_counts.values,
                labels=[f"{pct:.1f}%" for pct in percentages],
                colors=colors_type,
                startangle=90,
                textprops={'color': pie_label_color_inside, 'fontsize': 8, 'weight': 'bold'}
            )

            legend_labels = [f"{name} ({count})" for name, count in type_counts.items()]
            ax_type.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                          fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

            ax_type.set_facecolor('none')
            fig_type.patch.set_alpha(0.0)

            st.pyplot(fig_type, use_container_width=False)
            plt.close(fig_type)

    col_btn_l, col_btn_c, col_btn_r = st.columns([1, 2, 1])
    with col_btn_c:
        st.markdown("<div style='margin-top: 70px;'></div>", unsafe_allow_html=True)
        st.button("駐外人員統計", key="btn_stats", use_container_width=True, on_click=switch_page, args=('stats',))
        st.markdown("<div style='margin-top: 50px;'></div>", unsafe_allow_html=True)
        st.button("駐外人員搜尋", key="btn_search", use_container_width=True, on_click=switch_page, args=('search',))

def search_page():
    if 'show_filter' not in st.session_state:
        st.session_state.show_filter = False
    if 'run_filter' not in st.session_state:
        st.session_state.run_filter = False
    if 'filtered_results' not in st.session_state:
        st.session_state.filtered_results = pd.DataFrame()

    with st.container():
        st.markdown("<div class='fixed-button-container'>", unsafe_allow_html=True)
        st.button("⌂ 返回首頁", key="btn_fixed_back_home", on_click=switch_page, args=('home',))
        st.markdown("</div>", unsafe_allow_html=True)

    display_title("駐外人員搜尋")
    st.markdown("<div style='margin-top: 10px;'></div>", unsafe_allow_html=True)

    df_staff, df_scores = load_data()

    col_filter_1, col_filter_2, col_filter_3 = st.columns([1, 0.5, 1])

    with col_filter_2:
        if st.button("⚙️ 篩選器", key="btn_filter", use_container_width=True, on_click=toggle_filter, args=(True,)):
            st.rerun()

    if st.session_state.run_filter:
        if not df_staff.empty:
            st.session_state.filtered_results = apply_filters(df_staff, st.session_state.filters)
            st.session_state.run_filter = False
            st.success(f"篩選完成！共找到 {len(st.session_state.filtered_results)} 筆記錄。")
        else:
            st.error("無法執行篩選：統整表資料為空。")

    if not st.session_state.filtered_results.empty:
        with st.expander("✅ 篩選結果 (點擊展開或收合)", expanded=True):
            for index, row in st.session_state.filtered_results.iterrows():
                try:
                    name = row['員工姓名']
                except KeyError:
                    try:
                        name = row['姓名']
                    except KeyError:
                        st.error("統整表缺少 '員工姓名' 或 '姓名' 欄位。")
                        continue

                st.markdown(f"#### 👤 {name} 的人員資料")
                st.dataframe(pd.DataFrame(row).T, hide_index=True, use_container_width=True)
                if not df_scores.empty:
                    draw_score_chart(df_scores, name)
                else:
                    st.info("無法顯示考核分數圖：考核表資料缺失或讀取失敗。")
                st.markdown("---")

    if st.session_state.show_filter:
        show_filter_sidebar()

    st.markdown("<div style='margin-top: 50px;'></div>", unsafe_allow_html=True)

def stats_page():
    text_color, pie_label_color_inside, pie_legend_facecolor, _ = get_theme_colors()

    if 'show_filter' not in st.session_state:
        st.session_state.show_filter = False
    if 'run_filter' not in st.session_state:
        st.session_state.run_filter = False
    if 'filtered_results' not in st.session_state:
        st.session_state.filtered_results = pd.DataFrame()
    if 'selected_country' not in st.session_state:
        st.session_state.selected_country = None

    st.markdown("<div class='fixed-button-container'>", unsafe_allow_html=True)
    st.button("⌂ 返回首頁", key="btn_fixed_back_home_stats", on_click=switch_page, args=('home',))
    st.markdown("</div>", unsafe_allow_html=True)

    display_title("駐外人員統計")

    df_staff, df_scores = load_data()

    if df_staff.empty:
        st.error("無法載入人員資料。請檢查統整表檔案是否已正確上傳。")
        return

    country_list = ['烏克蘭', '瓜地馬拉', '薩爾瓦多', '貝里斯', '海地', '聖文森', '聖克里斯多福及尼維斯', '聖露西亞', '巴布亞紐幾內亞', '泰國', '印尼', '吐瓦魯', '帛琉', '馬紹爾', '斐濟', '菲律賓', '土耳其', '巴林', '沙烏地阿拉伯', '約旦', '立陶宛', '史瓦帝尼', '索馬利蘭', '坦尚尼亞', '巴拉圭']

    st.markdown("<div style='margin-top: 20px;'></div>", unsafe_allow_html=True)
    col_select1, col_select2, col_select3 = st.columns([1, 2, 1])
    with col_select2:
        selected_country_dropdown = st.selectbox(
            "🌍 選擇國家查看統計資料",
            ["請選擇..."] + country_list,
            key="country_selector"
        )
        if selected_country_dropdown != "請選擇...":
            st.session_state.selected_country = selected_country_dropdown

    country_mapping = {
        '烏克蘭': 'Ukraine', '瓜地馬拉': 'Guatemala', '薩爾瓦多': 'El Salvador',
        '貝里斯': 'Belize', '海地': 'Haiti', '聖文森': 'Saint Vincent and the Grenadines',
        '聖克里斯多福及尼維斯': 'Saint Kitts and Nevis', '聖露西亞': 'Saint Lucia',
        '巴布亞紐幾內亞': 'Papua New Guinea', '泰國': 'Thailand', '印尼': 'Indonesia',
        '吐瓦魯': 'Tuvalu', '帛琉': 'Palau', '馬紹爾': 'Marshall Islands',
        '斐濟': 'Fiji', '菲律賓': 'Philippines', '土耳其': 'Turkey',
        '巴林': 'Bahrain', '沙烏地阿拉伯': 'Saudi Arabia', '約旦': 'Jordan',
        '立陶宛': 'Lithuania', '史瓦帝尼': 'Eswatini', '索馬利蘭': 'Somaliland',
        '坦尚尼亞': 'Tanzania', '巴拉圭': 'Paraguay'
    }

    country_stats = {}
    for chinese_name, english_name in country_mapping.items():
        if '駐在國' not in df_staff.columns:
            st.error("統整表中缺少 '駐在國' 欄位，無法進行國家統計。")
            return

        country_df = df_staff[df_staff['駐在國'] == chinese_name].copy()

        if len(country_df) > 0:
            missing_names = country_df['員工姓名'].isna().sum()
            valid_df = country_df[country_df['員工姓名'].notna()].copy()
            total_staff = len(valid_df)

            stats = {
                'chinese_name': chinese_name,
                'english_name': english_name,
                'total': total_staff,
                'missing_names': missing_names,
                'has_missing_data': False
            }

            if total_staff > 0:
                gender_counts = valid_df['性別'].value_counts()
                stats['gender_counts'] = {'男': gender_counts.get('男', 0), '女': gender_counts.get('女', 0)}

                ages = valid_df['年齡'].dropna()
                stats['avg_age'] = ages.mean() if len(ages) > 0 else None

                if valid_df['性別'].isna().any() or valid_df['年齡'].isna().any():
                    stats['has_missing_data'] = True

                stats['types'] = valid_df['種類'].value_counts().to_dict()
                stats['titles'] = valid_df['職稱'].value_counts().to_dict()
                stats['levels'] = valid_df['職等'].value_counts().to_dict()

                country_stats[chinese_name] = stats

    map_data = []
    for chinese_name, stats in country_stats.items():
        map_data.append({
            'country': stats['english_name'],
            'chinese_name': chinese_name,
            'staff_count': stats['total']
        })

    map_df = pd.DataFrame(map_data)

    import plotly.express as px

    if len(map_df) > 0:
        fig = px.choropleth(
            map_df,
            locations='country',
            locationmode='country names',
            color='staff_count',
            hover_name='chinese_name',
            hover_data={'country': False, 'staff_count': True, 'chinese_name': False},
            color_continuous_scale=['#90EE90', '#006400'],
            labels={'staff_count': '人員數量'}
        )

        if st.get_option("theme.base") == 'dark':
             map_bg_color = 'rgba(0,0,0,0)'
             map_land_color = 'rgb(50, 50, 50)'
        else:
             map_bg_color = 'rgba(0,0,0,0)'
             map_land_color = 'rgb(200, 200, 200)'

        fig.update_geos(
            showcountries=True,
            countrycolor="darkgray",
            showcoastlines=True,
            coastlinecolor="gray",
            projection_type="natural earth",
            bgcolor=map_bg_color
        )

        fig.update_layout(
            geo=dict(showframe=False, showland=True, landcolor=map_land_color, bgcolor=map_bg_color),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=600,
            margin={"r":0,"t":30,"l":0,"b":0},
            coloraxis_colorbar=dict(title="人員數量", tickfont=dict(color=text_color), titlefont=dict(color=text_color))
        )

        selected_points = st.plotly_chart(fig, use_container_width=True, on_select="rerun", key="country_map")

        if selected_points and 'selection' in selected_points and 'points' in selected_points['selection']:
            points = selected_points['selection']['points']
            if len(points) > 0:
                selected_location = points[0].get('location', None)
                for cn, en in country_mapping.items():
                    if en == selected_location:
                        st.session_state.selected_country = cn
                        break

        if st.session_state.selected_country and st.session_state.selected_country in country_stats:
            stats = country_stats[st.session_state.selected_country]

            st.markdown("---")

            header_html = f"<h2>📍 {stats['chinese_name']}"
            if stats['has_missing_data']:
                header_html += " <span style='color: red; font-size: 20px;'>⚠️</span>"
            header_html += "</h2>"
            st.markdown(header_html, unsafe_allow_html=True)

            if stats['missing_names'] > 0:
                st.warning(f"⚠️ 此國家有 {stats['missing_names']} 筆員工姓名資料為空 (NA)")
            if stats['has_missing_data']:
                st.info("ℹ️ 部分統計資料因缺失值已自動排除")

            col1, col2 = st.columns(2)
            with col1:
                st.metric("👥 人員數量", f"{stats['total']} 人")
            with col2:
                if stats['avg_age']:
                    st.metric("📅 平均年齡", f"{stats['avg_age']:.1f} 歲")
                else:
                    st.metric("📅 平均年齡", "無資料")

            st.markdown("---")

            chart_col1, chart_col2 = st.columns(2)

            with chart_col1:
                st.markdown("#### 👥 性別分佈 (Gender Distribution)")
                fig_gender, ax = plt.subplots(figsize=(1.8, 1.8))
                gender_data = stats['gender_counts']

                percentages = [v/stats['total']*100 for v in gender_data.values()]
                labels_inside = [f"{v} ({p:.1f}%)" for v, p in zip(gender_data.values(), percentages)]

                ax.pie(gender_data.values(), labels=labels_inside, autopct='', colors=['#4A90E2', '#E24A8D'],
                       startangle=90,
                       textprops={'color': pie_label_color_inside, 'fontsize': 9})

                legend_labels = [f"{k} ({v}人)" for k, v in gender_data.items()]
                ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                          fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

                ax.set_facecolor('none')
                fig_gender.patch.set_alpha(0.0)
                st.pyplot(fig_gender, use_container_width=True)
                plt.close(fig_gender)

                st.markdown("#### 💼 職稱分佈 (Title Distribution)")
                if stats['titles']:
                    fig_title, ax = plt.subplots(figsize=(1.8, 1.8))

                    percentages = [v/stats['total']*100 for v in stats['titles'].values()]
                    labels_inside = [f"{p:.1f}%" for p in percentages]

                    ax.pie(stats['titles'].values(), labels=labels_inside, autopct='', colors=plt.cm.Pastel1(range(len(stats['titles']))),
                           startangle=90, textprops={'color': pie_label_color_inside, 'fontsize': 8})

                    legend_labels = [f"{k} ({v}人)" for k, v in stats['titles'].items()]
                    ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                              fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

                    ax.set_facecolor('none')
                    fig_title.patch.set_alpha(0.0)
                    st.pyplot(fig_title, use_container_width=True)
                    plt.close(fig_title)

            with chart_col2:
                st.markdown("#### 📊 種類分佈 (Type Distribution)")
                if stats['types']:
                    fig_type, ax = plt.subplots(figsize=(1.8, 1.8))

                    percentages = [v/stats['total']*100 for v in stats['types'].values()]
                    labels_inside = [f"{p:.1f}%" for p in percentages]

                    ax.pie(stats['types'].values(), labels=labels_inside, autopct='', colors=plt.cm.Set3(range(len(stats['types']))),
                           startangle=90, textprops={'color': pie_label_color_inside, 'fontsize': 8})

                    legend_labels = [f"{k} ({v}人)" for k, v in stats['types'].items()]
                    ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                              fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

                    ax.set_facecolor('none')
                    fig_type.patch.set_alpha(0.0)
                    st.pyplot(fig_type, use_container_width=True)
                    plt.close(fig_type)

                st.markdown("#### 🎓 職等分佈 (Level Distribution)")
                if stats['levels']:
                    fig_level, ax = plt.subplots(figsize=(1.8, 1.8))

                    percentages = [v/stats['total']*100 for v in stats['levels'].values()]
                    labels_inside = [f"{p:.1f}%" for p in percentages]

                    ax.pie(stats['levels'].values(), labels=labels_inside, autopct='', colors=plt.cm.Pastel2(range(len(stats['levels']))),
                           startangle=90, textprops={'color': pie_label_color_inside, 'fontsize': 8})

                    legend_labels = [f"{k} ({v}人)" for k, v in stats['levels'].items()]
                    ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1),
                              fontsize=8, framealpha=0.8, facecolor=pie_legend_facecolor, labelcolor=text_color)

                    ax.set_facecolor('none')
                    fig_level.patch.set_alpha(0.0)
                    st.pyplot(fig_level, use_container_width=True)
                    plt.close(fig_level)

    st.markdown("<div style='margin-top: 50px;'></div>", unsafe_allow_html=True)

if 'page' not in st.session_state:
    st.session_state.page = 'home'

if st.session_state.page == 'home':
    home_page()
elif st.session_state.page == 'search':
    search_page()
elif st.session_state.page == 'stats':
    stats_page()
"""

# 替換變數
try:
    modified_template = main_app_template.replace('__EXCEL_FILE_PATH__', f"'{EXCEL_FILE}'")
    modified_template = modified_template.replace('__SCORES_FILE_PATH__', f"'{SCORES_FILE}'")
    modified_template = modified_template.replace('__DRIVE_ROOT_PATH__', f"'{DRIVE_ROOT}'")
    modified_template = modified_template.replace('__LOGO_FILE_PATH__', f"'{LOGO_FILE}'")

    final_app_code = modified_template

    with open('main_app.py', 'w', encoding='utf-8') as f:
        f.write(final_app_code)
    print("✅ main_app.py 寫入完成。")
except Exception as e:
    print(f"❌ 寫入 main_app.py 時發生錯誤: {e}")
    sys.exit()

# --- 測試 Streamlit 語法 ---
print("\n=== 測試 Streamlit 語法 ===")
test_result = subprocess.run(
    ["python", "-m", "py_compile", "main_app.py"],
    capture_output=True,
    text=True
)

if test_result.returncode != 0:
    print("❌ Python 語法錯誤:")
    print(test_result.stderr)
    sys.exit()
else:
    print("✅ Python 語法檢查通過")

# --- 清理舊的 Streamlit 進程 ---
STREAMLIT_PORT = 8501

def cleanup():
    print("\n--- 服務已停止清理中 ---")
    try:
        ngrok.kill()
    except Exception:
        pass
    try:
        subprocess.run(["pkill", "-9", "-f", f"streamlit run {DRIVE_ROOT}/main_app.py"], check=False)
    except Exception:
        pass

signal.signal(signal.SIGINT, lambda s, f: cleanup() and sys.exit(0))

cleanup()
time.sleep(2)

# --- 啟動 Streamlit ---
streamlit_process = None

def start_streamlit():
    global streamlit_process
    print(f"\n✅ Streamlit 服務已在背景啟動 (Port: {STREAMLIT_PORT})...")
    try:
        streamlit_process = subprocess.Popen(
            ["python", "-m", "streamlit", "run", "main_app.py",
             "--server.port", str(STREAMLIT_PORT),
             "--server.enableCORS", "False",
             "--server.enableXsrfProtection", "False",
             "--browser.gatherUsageStats", "False"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        streamlit_process.wait()

    except Exception as e:
        print(f"❌ Streamlit 服務執行失敗: {e}")

streamlit_thread = threading.Thread(target=start_streamlit)
streamlit_thread.start()

print("⏳ 等待 Streamlit 服務初始化 (10 秒)...")
time.sleep(10)

# --- 啟動 Ngrok ---
print("\n🚀 正在啟動 Ngrok Tunnel...")

try:
    tunnel = ngrok.connect(STREAMLIT_PORT, 'http')
    public_url = tunnel.public_url

    if public_url:
        print("\n" + "="*60)
        print("✅ 網頁應用程式已部署成功 (使用 Ngrok)！")
        print("="*60)
        print(f"\n🌐 請點擊以下網址進入網頁：\n")
        print(f"   {public_url}")
        print(f"\n⚠️  重要提醒：")
        print("   • 保持此程式執行中，不要中斷！")
        print("   • 若要停止服務，請按 Ctrl+C 或停止此 Colab Cell")
        print("="*60 + "\n")

        # 持續運行
        while streamlit_thread.is_alive():
            time.sleep(1)

    else:
        print("\n❌ 部署失敗：無法取得 Ngrok URL。")

except Exception as e:
    print(f"\n❌ Ngrok 啟動失敗")
    print(f"錯誤訊息: {e}")
    print("\n可能的解決方法：")
    print("1. 確認 Ngrok 認證金鑰是否正確")
    print("2. 檢查網路連線狀態")
    print("3. 確認 Streamlit 服務是否正常啟動")

finally:
    cleanup()


開始設定 Matplotlib 中文字型...
✅ Matplotlib 快取檔案已清除 (1 個)。

正在下載並設定 Ngrok...
✅ Ngrok 已存在。
✅ Ngrok 認證金鑰已設定。
✅ 已清除所有舊的 Ngrok 服務。
✅ Drive 已掛載。
目前工作目錄已切換到: /content/drive/My Drive/駐外人員名冊colab

--- 檔案存在性檢查 ---
✅ 統整表 檔案已找到: /content/drive/My Drive/駐外人員名冊colab/全部的駐外人員統整表(滾動式更新-閱覽用).xlsx
✅ 考核表 檔案已找到: /content/drive/My Drive/駐外人員名冊colab/駐外歷年考核成績(滾動式更新) 未包含新員工.xlsx
✅ LOGO圖 檔案已找到: /content/drive/My Drive/駐外人員名冊colab/ICDF LOGO.png

開始寫入 main_app.py...
✅ main_app.py 寫入完成。

=== 測試 Streamlit 語法 ===
✅ Python 語法檢查通過

--- 服務已停止清理中 ---

✅ Streamlit 服務已在背景啟動 (Port: 8501)...
⏳ 等待 Streamlit 服務初始化 (10 秒)...

🚀 正在啟動 Ngrok Tunnel...

✅ 網頁應用程式已部署成功 (使用 Ngrok)！

🌐 請點擊以下網址進入網頁：

   https://callie-nontubular-mellisa.ngrok-free.dev

⚠️  重要提醒：
   • 保持此程式執行中，不要中斷！
   • 若要停止服務，請按 Ctrl+C 或停止此 Colab Cell



In [ ]:
# @title
# ========================================
# 終極修復腳本 - 一次解決所有問題
# ========================================

import subprocess
import time
import os
import sys

print("🔧 開始修復...")

# 1. 完全清理
print("\n[1/6] 清理舊進程...")
!pkill -9 -f streamlit
!pkill -9 -f ngrok
time.sleep(3)

# 2. 確認工作目錄
print("[2/6] 設定工作目錄...")
DRIVE_ROOT = '/content/drive/My Drive/駐外人員名冊colab'
os.chdir(DRIVE_ROOT)
print(f"✅ 工作目錄: {os.getcwd()}")

# 3. 安裝必要套件
print("[3/6] 檢查套件...")
!pip install -q streamlit pandas matplotlib plotly openpyxl Pillow pyngrok

# 4. 修改啟動方式（關鍵！）
print("[4/6] 啟動 Streamlit...")

# ⭐ 關鍵修改：加入 --server.fileWatcherType none
streamlit_process = subprocess.Popen([
    sys.executable, "-m", "streamlit", "run", "main_app.py",
    "--server.port", "8501",
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.fileWatcherType", "none",  # ⭐ 這行很重要！
    "--logger.level", "error"  # 減少日誌輸出
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# 等待更久（讓 Streamlit 完全啟動）
print("   等待 Streamlit 初始化...")
time.sleep(30)  # ⭐ 增加到 30 秒

# 檢查是否活著
if streamlit_process.poll() is not None:
    print("❌ Streamlit 崩潰了！")
    print("錯誤訊息:")
    print(streamlit_process.stderr.read())
else:
    print("✅ Streamlit 正在運行")

# 5. 啟動 Ngrok
print("[5/6] 啟動 Ngrok...")
from pyngrok import ngrok, conf

ngrok.kill()
time.sleep(2)

conf.get_default().ngrok_path = "/content/ngrok"
tunnel = ngrok.connect(8501, "http")
public_url = tunnel.public_url

print("\n" + "="*60)
print("✅ 部署成功！")
print("="*60)
print(f"\n🌐 網址: {public_url}")
print("\n⚠️ 請等待 30-60 秒後再訪問")
print("   （Streamlit 正在載入 Excel 資料）")
print("="*60)

# 6. 持續監控
print("\n[6/6] 監控服務狀態...")
print("（按 Ctrl+C 停止）\n")

try:
    while True:
        time.sleep(10)

        # 檢查 Streamlit
        if streamlit_process.poll() is not None:
            print("❌ Streamlit 已停止！")
            break

        # 檢查 Ngrok
        tunnels = ngrok.get_tunnels()
        if not tunnels:
            print("❌ Ngrok 已斷線！")
            break

        print(f"✅ 運行中 - {time.strftime('%H:%M:%S')}", end='\r')

except KeyboardInterrupt:
    print("\n停止服務...")
finally:
    streamlit_process.terminate()
    ngrok.kill()
    print("✅ 已清理")

🔧 開始修復...

[1/6] 清理舊進程...
[2/6] 設定工作目錄...
✅ 工作目錄: /content/drive/My Drive/駐外人員名冊colab
[3/6] 檢查套件...
[4/6] 啟動 Streamlit...
   等待 Streamlit 初始化...
✅ Streamlit 正在運行
[5/6] 啟動 Ngrok...

✅ 部署成功！

🌐 網址: https://callie-nontubular-mellisa.ngrok-free.dev

⚠️ 請等待 30-60 秒後再訪問
   （Streamlit 正在載入 Excel 資料）

[6/6] 監控服務狀態...
（按 Ctrl+C 停止）


停止服務...
✅ 已清理
